In [1]:
import subprocess
import shlex
import os
import glob
import re
import random

from subprocess import PIPE
from pathlib import Path

In [2]:
def submit_independent_isolated_landscape(
    sample, repeats=10, walltime=12, speciation=1e-6, partitions=10, memory=32, output='./STDIN'
):
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    ISOLATED_REPLAY_FILE_PATTERN = re.compile(fr"limit\.(\d+)(?:\[\]\.pbs)?\.e\d+")
    ISOLATED_PARTITIONS_FILE_PATTERN = re.compile(fr"limit\.isolated\.e(\d+)\.\d+")
    
    successful_replay = set()
    erroneous_replay = set()
    
    successful_partition = set()
    erroneous_partition = set()
    
    for path in output.parent.iterdir():
        replay_match = ISOLATED_REPLAY_FILE_PATTERN.match(path.name)
        partition_match = ISOLATED_PARTITIONS_FILE_PATTERN.match(path.name)
        
        if replay_match is not None:
            with open(path) as file:
                content = file.read()

            if content == '':
                successful_replay.add(replay_match.group(1))
            else:
                print(path, content)
                erroneous_replay.add(replay_match.group(1))
        elif partition_match is not None:
            with open(path) as file:
                content = file.read()

            if content == '':
                successful_partition.add(partition_match.group(1))
            else:
                erroneous_partition.add(partition_match.group(1))
    
    erroneous = erroneous_replay | erroneous_partition | (successful_partition - successful_replay)
    successful = successful_replay - erroneous
    
    for error in erroneous:
        for error_path in glob.iglob(f"{glob.escape(output)}.{error}.[oe]*"):
            Path(error_path).unlink()
            
        for error_path in glob.iglob(f"{glob.escape(output)}.{error}[[][]].pbs.[oe]*"):
            Path(error_path).unlink()
            
        for error_path in glob.iglob(f"{glob.escape(output)}.isolated.[oe]{error}.*"):
            Path(error_path).unlink()
        
    repeats = repeats - len(successful)
    
    if repeats < 1:
        return None
    
    print(f"{repeats} x {output}")
    
    walltime = "{:02d}".format(walltime)
    
    results = []
    
    for _ in range(repeats):
        seed = random.getrandbits(64)
        
        config = fr"""
        #PBS -lselect={1}:ncpus={1}:mem={16}gb
        #PBS -J {0}-{max(partitions - 1, 0)}
        #PBS -lwalltime={walltime}:00:00

        $HOME/necsim-rust/target-base/release/rustcoalescence simulate '(
            speciation: {speciation},
            sample: {sample},
            seed: {seed},

            algorithm: Independent(
                delta_t: {2.0},
                step_slice: {10},
                dedup_cache: Relative(factor: {1.0}),
                parallelism_mode: IsolatedLandscape(
                    event_slice:{100000000},
                    partition: Partition(rank:'$PBS_ARRAY_INDEX',partitions:{partitions})
                ),
            ),

            scenario: SpatiallyExplicit(
                habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size8/habitat.tif",
                dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size8/dispersal.tif",
            ),

            log: "'$EPHEMERAL'/'$(echo $PBS_JOBID | sed "s:\[[0-9]\+\]:\[\]:g")'/event_log/'$PBS_ARRAY_INDEX'",

            reporters: [
                Plugin(
                    library: "'$HOME'/necsim-rust/target-base/release/deps/libnecsim_plugins_common.so",
                    reporters: [Execution(), Biodiversity()],
                ),
            ],
        )'
        """
    
        cwd = os.getcwd()
        os.chdir(output.parent)

        # Submit the simulation batch
        pbs_jobid = subprocess.run(shlex.split(
            f"{os.environ['HOME']}/qsubbuf/target/release/qsubbuf -N {output.name}.isolated"
        ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
        
        pbs_jobid_folder = pbs_jobid.replace("[]", "[[][]]")
        
        config = fr"""
        #PBS -lselect={1}:ncpus={1}:mem={memory}gb
        #PBS -lwalltime={walltime}:00:00
        #PBS -W depend=afterok:{pbs_jobid}

        $HOME/necsim-rust/target-replay/release/rustcoalescence replay '(
            logs: [
                "'$EPHEMERAL'/{pbs_jobid_folder}/event_log/*/*/*"
            ],
            
            mode: WarnOnly,

            reporters: [
                Plugin(
                    library: "'$HOME'/necsim-rust/target-replay/release/deps/libnecsim_plugins_common.so",
                    reporters: [Execution(), Biodiversity(), Counter()],
                ),
            ],
        )'
        
        rm -rf $EPHEMERAL/{pbs_jobid_folder}/event_log
        """
        
        # Submit the simulation batch
        results.append(subprocess.run(shlex.split(
            f"{os.environ['HOME']}/qsubbuf/target/release/qsubbuf -N {output.name}.{pbs_jobid.replace('[]', '')}"
        ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip())

        os.chdir(cwd)
    
    return results

In [3]:
for sample, partitions, output in [(1.0 / 71.0, 10, "1e9"), (1.0 / 7.1, 100, "1e10"), (1.0, 710, "7.1e10")]:
    submit_independent_isolated_landscape(
        sample, repeats=10, walltime=12, speciation=1e-6, partitions=partitions, memory=32, output=f"{output}/limit"
    )